In [1]:
%env TF_DISABLE_MLC=1

env: TF_DISABLE_MLC=1


In [2]:
from mtg.obj.games import Games
from mtg.ml.models import DeckBuilder
from mtg.ml.trainer import Trainer
from mtg.utils.display import print_deck
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
data_file = '/Users/a10889158/personal/mtg/data/game_data_public.KHM.PremierDraft.csv'

In [4]:
%%time
games = Games(data_file,'khm')

/Users/a10889158/miniforge3/envs/personal/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


CPU times: user 9.13 s, sys: 2.9 s, total: 12 s
Wall time: 13.3 s


In [5]:
decks = games.get_decks()

In [6]:
idxs = np.arange(len(decks))
train_idxs = np.random.choice(idxs,int(len(idxs) * 0.8),replace=False)
test_idxs = np.asarray(list(set(idxs.flatten()) - set(train_idxs.flatten())))

In [7]:
train_decks = decks[train_idxs]
test_decks = decks[test_idxs]

In [8]:
land_idxs = games.cards[games.cards['type_line'].apply(lambda x: "land" in x.lower())]['idx'].to_numpy()

In [9]:
model = DeckBuilder(train_decks.shape[1],land_idxs)
model.compile(
    optimizer=tf.optimizers.Adam(lr=0.0001),
    loss_function=tf.losses.mean_squared_error,
    reg_coef=1.0,
    constraint_coef=0.1,
)
t = Trainer(train_decks, train_decks, model)
t.train(25, batch_size=64)

Epoch 25/25: 100%|██████████| 718/718 [00:08<00:00, 85.74Batch/s, loss=0.0643]


In [10]:
import pandas as pd
import numpy as np
def compare(self, deck, card_df, sort_by=['cmc','type_line']):
    deck = np.expand_dims(deck, 0)
    model_output = self.__call__(deck)
    built = tf.squeeze(self.round_to_deck(model_output))
    fake_sb = tf.squeeze(self.noisy_decks - deck)
    deck = tf.squeeze(deck)
    df = pd.DataFrame(columns=["name","real deck","fake sideboard","predicted deck"])
    card_df = card_df.sort_values(by=sort_by)
    for card_idx, card_name in card_df[['idx','name']].to_numpy():
        row_dict = {col:None for col in df.columns}
        row_dict['name'] = card_name
        sb_count = fake_sb[card_idx]
        n_found = 0
        if sb_count > 0:
            row_dict['fake sideboard'] = sb_count.numpy()
            n_found += 1
        deck_count = deck[card_idx]
        if deck_count > 0:
            row_dict['real deck'] = deck_count.numpy()
            n_found += 1
        pred_count = built[card_idx]
        if pred_count > 0:
            row_dict['predicted deck'] = pred_count.numpy()
            n_found += 1
        if n_found > 0:
            df = df.append(row_dict, ignore_index=True)  
    return df      

In [17]:
compare(model, test_decks[10], games.cards)

,name,real deck,fake sideboard,predicted deck
0,mountain,5,NaN,5.0
1,plains,8,NaN,8.0
2,snow-covered mountain,1,NaN,1.0
3,snow-covered swamp,1,NaN,1.0
4,barkchannel pathway // tidechannel pathway,None,1.0,None
5,shimmerdrift vale,2,None,2.0
6,alpine meadow,None,1.0,1.0
7,snowfield sinkhole,None,1.0,None
8,usher of the fallen,1,None,1.0
9,ascendant spirit,None,1.0,None
